This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

esRJKHtAFczu5B_LMQVM


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [14]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url='https://www.quandl.com/api/v3/datasets/FSE/AFX_X'
r=requests.get(url)
json_data=r.json()
print(json_data['dataset']['data'][0])

['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None]


In [238]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [236]:
#~1~#
d=json_data['dataset']
data_2017=[]
for data in d['data']:
    if '2017' in str(data[0]):
        data_2017.append(data)

#~2~#
columns=['Open','High','Low','Close','Change','Traded Volume','Turnover',\
 'Last Price of the Day','Daily Traded Units','Daily Turnover']

for data in data_2017:
    dict_2017[data[0]]={}
    for i in range(len(columns)):
         dict_2017[data[0]].update({columns[i]:data[1:][i]})
            
#~3~#
highest_open=0.0
lowest_open=100
for date in dict_2017:
    s=dict_2017[date]['Open']
    if type(s)==float:
        if s>highest_open:
            highest_open=s
        if s<lowest_open:
            lowest_open=s
print('~*2017 NASDAQ Prices*~')       
print('The highest opening price in this year is: '+str(highest_open)+'.')
print('The lowest opening price is: ' +str(lowest_open)+'.')

#~4~#
for date in dict_2017:
    dict_2017[date].pop('Change') #delete existing key cuz None everywhere

highest_change_day=0.0
for date in dict_2017:
    high_price=dict_2017[date]['High']
    low_price=dict_2017[date]['Low']
    #update new 'Change' column where both values exist
    if type(high_price)==float and type(low_price)==float:
        change_price=round(high_price-low_price,3)
        dict_2017[date].update({'Change':change_price})
        #find max change in price over a day
        if change_price > highest_change_day:
            highest_change_day=change_price
            highest_change_date=date
    
print('The largest change in price over a day is: '+ \
      str(highest_change_day)+ ', on '+str(date)+'.')

#~5~#
two_days=[]
date_strs=[]
for date in dict_2017.keys():
    date_str=date[0:4]+date[5:7]+date[8:]
    date_strs.append(date_str)
for date in date_strs:
    earlier_date=int(date)-2
    if str(earlier_date) in date_strs:
        two_days.append(date)
        
def dashed(d):
    return d[0:4]+'-'+d[4:6]+'-'+d[6:]

highest_change_2days=0.0
for date in two_days:
    closing_after=dict_2017[dashed(str(date))]['Close']
    closing_before=dict_2017[dashed(str((int(date)-2)))]['Close']
    change_in_price=round(closing_after-closing_before,3)
    if change_in_price > highest_change_2days:
        highest_change_2days=change_in_price
        highest_change_2days_after=dashed(str(date))
        highest_change_2days_before=dashed(str((int(date)-2)))

print('The largest change in price over 2 days is: ' + str(highest_change_2days)\
      +' from '+ str(highest_change_2days_before)+' to '+\
      str( highest_change_2days_after)+'.')

#~6~#
traded_vol_2017=[]
for date in dict_2017:
    trade_vol=dict_2017[date]['Traded Volume']
    if type(trade_vol==float):
        traded_vol_2017.append(trade_vol)
print('The average daily trading volume is: ' \
      +str(sum(traded_vol_2017)/len(traded_vol_2017))+'.')

#~7~#
def medianize(lst):
    l=sorted(lst)
    if len(l)%2==0:
        med_pos=int(len(l)/2)
        median=(l[med_pos]+l[med_pos-1])/2
        return median
    elif len(l)%2==1:
        med_pos=int((len(l)+1)/2)
        median=l[med_pos]
        return median

print('The median trading volume is: '+str(medianize(traded_vol_2017))+'.')

~*2017 NASDAQ Prices*~
The highest opening price in this year is: 53.11.
The lowest opening price is: 34.0.
The largest change in price over a day is: 2.81, on 2017-01-02.
The largest change in price over 2 days is: 2.13 from 2017-09-27 to 2017-09-29.
The average daily trading volume is: 89124.33725490196.
The median trading volume is: 76600.0.
